<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/Step1_mat_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import scipy.io as spio
import numpy as np


# EEGNet-specific imports
from EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt

In [2]:
x= 1         
fName = 'parsed_A0' + str(x) + 'T.mat'  # Load Data
print(fName)
mat = spio.loadmat(fName)
X = mat['cleanRawEEGData']
y = mat['cleanClassLabels']

parsed_A01T.mat


In [3]:
print(np.shape(X))
print(np.shape(y))

(273, 22, 1500)
(273, 1)


In [4]:
# take 50/25/25 percent of the data to train/validate/test
X_train      = X[0:150,:,500:1000]
Y_train      = y[0:150]
X_validate   = X[151:200,:,500:1000]
Y_validate   = y[151:200]
X_test       = X[201:,:,500:1000]
Y_test       = y[201:]

print(np.shape(X_train))
print(np.shape(Y_train))
print(np.shape(X_validate))
print(np.shape(Y_validate))
print(np.shape(X_test))
print(np.shape(Y_test))

(150, 22, 500)
(150, 1)
(49, 22, 500)
(49, 1)
(72, 22, 500)
(72, 1)


In [0]:

# convert labels to one-hot encodings.
Y_train      = np_utils.to_categorical(Y_train-1)
Y_validate   = np_utils.to_categorical(Y_validate-1)
Y_test       = np_utils.to_categorical(Y_test-1)

In [6]:
kernels, chans, samples = 1, 22, 500

# convert data to NCHW (trials, kernels, channels, samples) format. Data 
# contains 22 channels and 500 time-points. Set the number of kernels to 1.
X_train      = X_train.reshape(X_train.shape[0], kernels, chans, samples)
X_validate   = X_validate.reshape(X_validate.shape[0], kernels, chans, samples)
X_test       = X_test.reshape(X_test.shape[0], kernels, chans, samples)
   
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (150, 1, 22, 500)
150 train samples
72 test samples


In [0]:
#X_train= np.squeeze(X_train)

In [7]:
# configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
# model configurations may do better, but this is a good starting point)
model = EEGNet(nb_classes = 4, Chans = 22, Samples = 500, 
             dropoutRate = 0.5, kernLength = 25, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')




Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# compile the model and set the optimizers
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics = ['accuracy'])

In [0]:
# count number of parameters in the model
numParams    = model.count_params()    

# set a valid path for your system to record model checkpoints
checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
                               save_best_only=True)

In [10]:
# the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
# the weights all to be 1
class_weights = {0:1, 1:1, 2:1, 3:1}

################################################################################
# fit the model. Due to very small sample sizes this can get
# pretty noisy run-to-run, but most runs should be comparable to xDAWN + 
# Riemannian geometry classification (below)
################################################################################
fittedModel = model.fit(X_train, Y_train, batch_size = 16, epochs = 300, 
                        verbose = 2, validation_data=(X_validate, Y_validate),
                        callbacks=[checkpointer], class_weight = class_weights)

Train on 150 samples, validate on 49 samples
Epoch 1/300

Epoch 00001: val_loss improved from inf to 1.38328, saving model to /tmp/checkpoint.h5
150/150 - 4s - loss: 1.4054 - acc: 0.2667 - val_loss: 1.3833 - val_acc: 0.2857
Epoch 2/300

Epoch 00002: val_loss improved from 1.38328 to 1.37705, saving model to /tmp/checkpoint.h5
150/150 - 0s - loss: 1.3608 - acc: 0.3200 - val_loss: 1.3770 - val_acc: 0.2857
Epoch 3/300

Epoch 00003: val_loss improved from 1.37705 to 1.37135, saving model to /tmp/checkpoint.h5
150/150 - 0s - loss: 1.3249 - acc: 0.4600 - val_loss: 1.3713 - val_acc: 0.3265
Epoch 4/300

Epoch 00004: val_loss improved from 1.37135 to 1.36256, saving model to /tmp/checkpoint.h5
150/150 - 0s - loss: 1.2909 - acc: 0.4867 - val_loss: 1.3626 - val_acc: 0.3878
Epoch 5/300

Epoch 00005: val_loss improved from 1.36256 to 1.35258, saving model to /tmp/checkpoint.h5
150/150 - 0s - loss: 1.2492 - acc: 0.4800 - val_loss: 1.3526 - val_acc: 0.3878
Epoch 6/300

Epoch 00006: val_loss improved 